In [1]:
import jax
import jax.numpy as jnp
import haiku as hk
import optax
import sys
from time import sleep
import json
import pexpect
import re

In [2]:
HOLPATH = "/home/sean/Documents/PhD/HOL4/HOL/bin/hol --maxheap=256"
TARGET_THEORIES = ["pred_set"] #["arithmetic"]#["list"] #["rich_list"] # ["integer"] #["arithmetic"] # ["rich_list"] #["pred_set"]

In [3]:
with open("typed_database.json") as f:
    database = json.load(f)

In [4]:
goals = [value[4] for key, value in database.items() if value[3] == "thm" and value[0] in TARGET_THEORIES]

In [5]:
goals

['∀(s :α -> bool). 𝕌(:α) ⊆ s ⇔ s = 𝕌(:α)',
 '𝕌(:α) ≠ (∅ :α -> bool)',
 'FUNSET 𝕌(:α) 𝕌(:β) = 𝕌(:α -> β)',
 '𝕌(:α -> bool) = POW 𝕌(:α)',
 '𝕌(:bool) = {T; F}',
 '∀(x :α). 𝕌(:α) x',
 '∀(x :α) (s :α -> bool). x ∈ s ∧ (∀(y :α). y ∈ s ⇒ x = y) ⇔ s = {x}',
 '(∀(s :α -> bool). 𝕌(:α) ∪ s = 𝕌(:α)) ∧ ∀(s :α -> bool). s ∪ 𝕌(:α) = 𝕌(:α)',
 '∀(s :α -> bool) (t :α -> bool) (u :α -> bool). s ∪ t ⊆ u ⇔ s ⊆ u ∧ t ⊆ u',
 '∀(s :α -> bool) (t :α -> bool) (u :α -> bool). s ∩ (t ∪ u) = s ∩ t ∪ s ∩ u',
 '∀(s :α -> bool). s ∪ s = s',
 '(∀(s :α -> bool). (∅ :α -> bool) ∪ s = s) ∧ ∀(s :α -> bool). s ∪ (∅ :α -> bool) = s',
 '(∀(s :α -> bool) (t :α -> bool). s ∪ (t DIFF s) = s ∪ t) ∧ ∀(s :α -> bool) (t :α -> bool). t DIFF s ∪ s = t ∪ s',
 '∀(s :α -> bool) (t :α -> bool). s ∪ (s DIFF t) = s',
 '(s :α -> bool) ⊆ (t :α -> bool) ⇒ s ∪ (t DIFF s) = t ∧ t DIFF s ∪ s = t',
 '∀(A :α -> bool) (B :α -> bool) (x :α). A ∪ B DELETE x = A DELETE x ∪ (B DELETE x)',
 'countable ((s :α -> bool) ∪ (t :α -> bool)) ⇔ countable s ∧ co

In [6]:
def get_polish(raw_goal):
        goal = construct_goal(raw_goal)
        process.sendline(goal.encode("utf-8"))
        process.expect("\r\n>")
        process.sendline("val _ = set_term_printer (HOLPP.add_string o pt);".encode("utf-8"))
        process.expect("\r\n>")
        process.sendline("top_goals();".encode("utf-8"))
        process.expect("val it =")
        process.expect([": goal list", ":\r\n +goal list"])

        polished_raw = process.before.decode("utf-8")
        polished_subgoals = re.sub("“|”","\"", polished_raw)
        polished_subgoals = re.sub("\r\n +"," ", polished_subgoals)

        # print("content:{}".format(subgoals))
        # exit()
        pd = eval(polished_subgoals)
        
        process.expect("\r\n>")
        process.sendline("drop();".encode("utf-8"))
        process.expect("\r\n>")
        process.sendline("val _ = set_term_printer default_pt;".encode("utf-8"))
        process.expect("\r\n>")

        data = [{"polished":{"assumptions": e[0][0], "goal":e[0][1]},
                 "plain":{"assumptions": e[1][0], "goal":e[1][1]}}
                for e in zip(pd, [([], raw_goal)])]
        return data # list(zip(pd, [([], raw_goal)]))f

def construct_goal(goal):
    s = "g " + "`" + goal + "`;"
    return s

In [7]:
process = pexpect.spawn(HOLPATH)

theories = ["listTheory", "bossLib"]

print("Importing theories...")
process.sendline("val _ = HOL_Interactive.toggle_quietdec();".encode("utf-8"))
for i in theories:
    process.sendline("open {};".format(i).encode("utf-8"))

Importing theories...


In [8]:

# remove built-in simp lemmas
print("Removing simp lemmas...")
process.sendline("delsimps [\"HD\", \"EL_restricted\", \"EL_simp_restricted\"];")
#process.sendline("delsimps {};".format(dels))
#process.sendline("delsimps {};".format(dels2))
# process.sendline("delsimps {};".format(dels3))
sleep(4)
# load utils
print("Loading modules...")
process.sendline("use \"helper.sml\";")
sleep(5)
# process.sendline("val _ = load \"Timeout\";")
print("Configuration done.")
process.expect('\r\n>')
# process.readline()
process.sendline("val _ = HOL_Interactive.toggle_quietdec();".encode("utf-8"))

# consumes hol4 head
process.expect('\r\n>')
#print (process.read())
# setup the goal
goal = goals[0]
                      
 # a pair of polished goal list and original goal list
fringe = get_polish(goal)

#scripts = []
goal = construct_goal(goal)
process.sendline(goal.encode("utf-8"))        
print("Initialization done. Main goal is:\n{}.".format(goal))


Removing simp lemmas...
Loading modules...
Configuration done.
Initialization done. Main goal is:
g `∀(s :α -> bool). 𝕌(:α) ⊆ s ⇔ s = 𝕌(:α)`;.


In [10]:
print (fringe)

[{'polished': {'assumptions': [], 'goal': '@ C$bool$ ! | Vs @ @ C$min$ = @ @ C$pred_set$ SUBSET C$pred_set$ UNIV Vs @ @ C$min$ = Vs C$pred_set$ UNIV'}, 'plain': {'assumptions': [], 'goal': '∀(s :α -> bool). 𝕌(:α) ⊆ s ⇔ s = 𝕌(:α)'}}]
